In [1]:
# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax, Adagrad
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.read_csv('../data/feature_selection_positive.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [3]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
def create_model(optimizer='rmsprop'):
    tf.set_random_seed(42)
    c=0.4
    # make keras model
    start = time.time()
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(1024, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    
    return model

In [5]:
model = KerasClassifier(
    build_fn=create_model, 
    verbose=2, 
    epochs=100, 
    batch_size=1000
)

In [6]:
p = [
        'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 
        'Adam', 'Adamax', 'Nadam'
]  # optimizers

params = dict(
    optimizer=p
             )

In [7]:
grid = GridSearchCV(
    estimator=model, 
    param_grid=params
)

In [8]:
grid_result = grid.fit(X_train, y_train_for_keras)

Epoch 1/100
 - 1s - loss: 1.8772 - acc: 0.2676
Epoch 2/100
 - 0s - loss: 1.9185 - acc: 0.2576
Epoch 3/100
 - 0s - loss: 1.8720 - acc: 0.2654
Epoch 4/100
 - 0s - loss: 1.8615 - acc: 0.2979
Epoch 5/100
 - 0s - loss: 1.8310 - acc: 0.2800
Epoch 6/100
 - 0s - loss: 1.7645 - acc: 0.2721
Epoch 7/100
 - 0s - loss: 1.7904 - acc: 0.2632
Epoch 8/100
 - 0s - loss: 1.8549 - acc: 0.2766
Epoch 9/100
 - 0s - loss: 1.7718 - acc: 0.2744
Epoch 10/100
 - 0s - loss: 1.8059 - acc: 0.2699
Epoch 11/100
 - 0s - loss: 1.7380 - acc: 0.2777
Epoch 12/100
 - 0s - loss: 1.7091 - acc: 0.2934
Epoch 13/100
 - 0s - loss: 1.6357 - acc: 0.2721
Epoch 14/100
 - 0s - loss: 1.7302 - acc: 0.2833
Epoch 15/100
 - 0s - loss: 1.7138 - acc: 0.2755
Epoch 16/100
 - 0s - loss: 1.6417 - acc: 0.2856
Epoch 17/100
 - 0s - loss: 1.6779 - acc: 0.2867
Epoch 18/100
 - 0s - loss: 1.6632 - acc: 0.2912
Epoch 19/100
 - 0s - loss: 1.6209 - acc: 0.3001
Epoch 20/100
 - 0s - loss: 1.6068 - acc: 0.2844
Epoch 21/100
 - 0s - loss: 1.5394 - acc: 0.3046
E

 - 0s - loss: 1.3435 - acc: 0.3583
Epoch 73/100
 - 0s - loss: 1.4086 - acc: 0.3326
Epoch 74/100
 - 0s - loss: 1.3141 - acc: 0.3942
Epoch 75/100
 - 0s - loss: 1.4125 - acc: 0.3673
Epoch 76/100
 - 0s - loss: 1.4007 - acc: 0.3337
Epoch 77/100
 - 0s - loss: 1.3820 - acc: 0.3595
Epoch 78/100
 - 0s - loss: 1.3152 - acc: 0.3796
Epoch 79/100
 - 0s - loss: 1.3994 - acc: 0.3404
Epoch 80/100
 - 0s - loss: 1.3444 - acc: 0.3841
Epoch 81/100
 - 0s - loss: 1.3490 - acc: 0.3774
Epoch 82/100
 - 0s - loss: 1.3674 - acc: 0.3415
Epoch 83/100
 - 0s - loss: 1.3337 - acc: 0.3751
Epoch 84/100
 - 0s - loss: 1.3487 - acc: 0.3651
Epoch 85/100
 - 0s - loss: 1.3340 - acc: 0.3964
Epoch 86/100
 - 0s - loss: 1.4121 - acc: 0.3449
Epoch 87/100
 - 0s - loss: 1.3410 - acc: 0.3617
Epoch 88/100
 - 0s - loss: 1.3522 - acc: 0.3505
Epoch 89/100
 - 0s - loss: 1.2949 - acc: 0.3841
Epoch 90/100
 - 0s - loss: 1.3367 - acc: 0.3886
Epoch 91/100
 - 0s - loss: 1.3324 - acc: 0.3908
Epoch 92/100
 - 0s - loss: 1.2986 - acc: 0.3998
Epoch

Epoch 44/100
 - 0s - loss: 0.3722 - acc: 0.8611
Epoch 45/100
 - 0s - loss: 0.3808 - acc: 0.8634
Epoch 46/100
 - 0s - loss: 0.3958 - acc: 0.8656
Epoch 47/100
 - 0s - loss: 0.4134 - acc: 0.8432
Epoch 48/100
 - 0s - loss: 0.3624 - acc: 0.8544
Epoch 49/100
 - 0s - loss: 0.3458 - acc: 0.8634
Epoch 50/100
 - 0s - loss: 0.2687 - acc: 0.9003
Epoch 51/100
 - 0s - loss: 0.2599 - acc: 0.8959
Epoch 52/100
 - 0s - loss: 0.2799 - acc: 0.8959
Epoch 53/100
 - 0s - loss: 0.2783 - acc: 0.8847
Epoch 54/100
 - 0s - loss: 0.3055 - acc: 0.8869
Epoch 55/100
 - 0s - loss: 0.2672 - acc: 0.8914
Epoch 56/100
 - 0s - loss: 0.2416 - acc: 0.9127
Epoch 57/100
 - 0s - loss: 0.2646 - acc: 0.8959
Epoch 58/100
 - 0s - loss: 0.2642 - acc: 0.9138
Epoch 59/100
 - 0s - loss: 0.2428 - acc: 0.9048
Epoch 60/100
 - 0s - loss: 0.3055 - acc: 0.9227
Epoch 61/100
 - 0s - loss: 0.1858 - acc: 0.9384
Epoch 62/100
 - 0s - loss: 0.2392 - acc: 0.9127
Epoch 63/100
 - 0s - loss: 0.2024 - acc: 0.9227
Epoch 64/100
 - 0s - loss: 0.2319 - acc:

Epoch 16/100
 - 0s - loss: 0.9738 - acc: 0.5906
Epoch 17/100
 - 0s - loss: 0.9269 - acc: 0.6107
Epoch 18/100
 - 0s - loss: 0.8843 - acc: 0.6007
Epoch 19/100
 - 0s - loss: 0.8736 - acc: 0.6253
Epoch 20/100
 - 0s - loss: 0.8316 - acc: 0.6398
Epoch 21/100
 - 0s - loss: 0.8828 - acc: 0.6331
Epoch 22/100
 - 0s - loss: 0.9127 - acc: 0.6298
Epoch 23/100
 - 0s - loss: 0.7997 - acc: 0.6767
Epoch 24/100
 - 0s - loss: 0.8578 - acc: 0.6477
Epoch 25/100
 - 0s - loss: 0.8016 - acc: 0.6734
Epoch 26/100
 - 0s - loss: 0.7637 - acc: 0.6745
Epoch 27/100
 - 0s - loss: 0.6518 - acc: 0.7371
Epoch 28/100
 - 0s - loss: 0.5790 - acc: 0.7796
Epoch 29/100
 - 0s - loss: 0.5455 - acc: 0.7875
Epoch 30/100
 - 0s - loss: 0.5756 - acc: 0.7729
Epoch 31/100
 - 0s - loss: 0.5798 - acc: 0.7864
Epoch 32/100
 - 0s - loss: 0.5824 - acc: 0.7808
Epoch 33/100
 - 0s - loss: 0.6871 - acc: 0.7461
Epoch 34/100
 - 0s - loss: 0.5741 - acc: 0.7841
Epoch 35/100
 - 0s - loss: 0.4875 - acc: 0.8043
Epoch 36/100
 - 0s - loss: 0.4192 - acc:

Epoch 87/100
 - 0s - loss: 0.1227 - acc: 0.9574
Epoch 88/100
 - 0s - loss: 0.1039 - acc: 0.9642
Epoch 89/100
 - 0s - loss: 0.1200 - acc: 0.9530
Epoch 90/100
 - 0s - loss: 0.1220 - acc: 0.9586
Epoch 91/100
 - 0s - loss: 0.0910 - acc: 0.9698
Epoch 92/100
 - 0s - loss: 0.1175 - acc: 0.9586
Epoch 93/100
 - 0s - loss: 0.1146 - acc: 0.9608
Epoch 94/100
 - 0s - loss: 0.1069 - acc: 0.9608
Epoch 95/100
 - 0s - loss: 0.1043 - acc: 0.9619
Epoch 96/100
 - 0s - loss: 0.1093 - acc: 0.9653
Epoch 97/100
 - 0s - loss: 0.1010 - acc: 0.9608
Epoch 98/100
 - 0s - loss: 0.1035 - acc: 0.9586
Epoch 99/100
 - 0s - loss: 0.1031 - acc: 0.9630
Epoch 100/100
 - 0s - loss: 0.1017 - acc: 0.9653
Epoch 1/100
 - 2s - loss: 1.8595 - acc: 0.2665
Epoch 2/100
 - 0s - loss: 3.5962 - acc: 0.2968
Epoch 3/100
 - 0s - loss: 6.7326 - acc: 0.2755
Epoch 4/100
 - 0s - loss: 5.1716 - acc: 0.3382
Epoch 5/100
 - 0s - loss: 4.4555 - acc: 0.2844
Epoch 6/100
 - 0s - loss: 2.2803 - acc: 0.3774
Epoch 7/100
 - 0s - loss: 2.4088 - acc: 0.315

...

Epoch 73/100
 - 0s - loss: 0.1845 - acc: 0.9362
Epoch 74/100
 - 0s - loss: 0.1806 - acc: 0.9328
Epoch 75/100
 - 0s - loss: 0.1730 - acc: 0.9261
Epoch 76/100
 - 0s - loss: 0.1700 - acc: 0.9429
Epoch 77/100
 - 0s - loss: 0.1592 - acc: 0.9440
Epoch 78/100
 - 0s - loss: 0.1701 - acc: 0.9429
Epoch 79/100
 - 0s - loss: 0.1386 - acc: 0.9552
Epoch 80/100
 - 0s - loss: 0.1531 - acc: 0.9474
Epoch 81/100
 - 0s - loss: 0.1539 - acc: 0.9530
Epoch 82/100
 - 0s - loss: 0.1545 - acc: 0.9451
Epoch 83/100
 - 0s - loss: 0.1361 - acc: 0.9485
Epoch 84/100
 - 0s - loss: 0.1191 - acc: 0.9496
Epoch 85/100
 - 0s - loss: 0.1291 - acc: 0.9552
Epoch 86/100
 - 0s - loss: 0.1255 - acc: 0.9619
Epoch 87/100
 - 0s - loss: 0.1190 - acc: 0.9597
Epoch 88/100
 - 0s - loss: 0.1307 - acc: 0.9518
Epoch 89/100
 - 0s - loss: 0.1247 - acc: 0.9630
Epoch 90/100
 - 0s - loss: 0.1160 - acc: 0.9619
Epoch 91/100
 - 0s - loss: 0.1090 - acc: 0.9586
Epoch 92/100
 - 0s - loss: 0.1340 - acc: 0.9586
Epoch 93/100
 - 0s - loss: 0.1065 - acc:

 - 0s - loss: 0.6367 - acc: 0.7458
Epoch 45/100
 - 0s - loss: 0.6302 - acc: 0.7413
Epoch 46/100
 - 0s - loss: 0.5967 - acc: 0.7514
Epoch 47/100
 - 0s - loss: 0.6226 - acc: 0.7346
Epoch 48/100
 - 0s - loss: 0.5887 - acc: 0.7559
Epoch 49/100
 - 0s - loss: 0.5892 - acc: 0.7514
Epoch 50/100
 - 0s - loss: 0.5833 - acc: 0.7615
Epoch 51/100
 - 0s - loss: 0.6040 - acc: 0.7559
Epoch 52/100
 - 0s - loss: 0.5427 - acc: 0.7783
Epoch 53/100
 - 0s - loss: 0.5477 - acc: 0.7660
Epoch 54/100
 - 0s - loss: 0.5691 - acc: 0.7637
Epoch 55/100
 - 0s - loss: 0.5226 - acc: 0.7951
Epoch 56/100
 - 0s - loss: 0.4874 - acc: 0.7996
Epoch 57/100
 - 0s - loss: 0.4996 - acc: 0.8108
Epoch 58/100
 - 0s - loss: 0.4681 - acc: 0.8141
Epoch 59/100
 - 0s - loss: 0.4706 - acc: 0.7996
Epoch 60/100
 - 0s - loss: 0.4580 - acc: 0.8231
Epoch 61/100
 - 0s - loss: 0.4555 - acc: 0.8231
Epoch 62/100
 - 0s - loss: 0.4571 - acc: 0.8141
Epoch 63/100
 - 0s - loss: 0.4536 - acc: 0.8208
Epoch 64/100
 - 0s - loss: 0.4271 - acc: 0.8219
Epoch

 - 0s - loss: 1.0703 - acc: 0.5515
Epoch 16/100
 - 0s - loss: 1.0830 - acc: 0.5313
Epoch 17/100
 - 0s - loss: 0.9946 - acc: 0.5626
Epoch 18/100
 - 0s - loss: 0.9920 - acc: 0.5548
Epoch 19/100
 - 0s - loss: 0.9699 - acc: 0.5772
Epoch 20/100
 - 0s - loss: 0.9974 - acc: 0.5638
Epoch 21/100
 - 0s - loss: 0.9561 - acc: 0.5414
Epoch 22/100
 - 0s - loss: 0.9278 - acc: 0.6007
Epoch 23/100
 - 0s - loss: 0.9109 - acc: 0.5928
Epoch 24/100
 - 0s - loss: 0.8792 - acc: 0.6085
Epoch 25/100
 - 0s - loss: 0.8575 - acc: 0.6107
Epoch 26/100
 - 0s - loss: 0.8586 - acc: 0.6119
Epoch 27/100
 - 0s - loss: 0.8524 - acc: 0.6219
Epoch 28/100
 - 0s - loss: 0.8290 - acc: 0.6286
Epoch 29/100
 - 0s - loss: 0.8233 - acc: 0.6342
Epoch 30/100
 - 0s - loss: 0.7834 - acc: 0.6633
Epoch 31/100
 - 0s - loss: 0.7915 - acc: 0.6600
Epoch 32/100
 - 0s - loss: 0.7833 - acc: 0.6555
Epoch 33/100
 - 0s - loss: 0.7648 - acc: 0.6488
Epoch 34/100
 - 0s - loss: 0.7336 - acc: 0.6823
Epoch 35/100
 - 0s - loss: 0.7304 - acc: 0.6924
Epoch

 - 0s - loss: 0.0968 - acc: 0.9664
Epoch 87/100
 - 0s - loss: 0.0940 - acc: 0.9675
Epoch 88/100
 - 0s - loss: 0.1082 - acc: 0.9675
Epoch 89/100
 - 0s - loss: 0.0875 - acc: 0.9675
Epoch 90/100
 - 0s - loss: 0.0979 - acc: 0.9597
Epoch 91/100
 - 0s - loss: 0.0855 - acc: 0.9675
Epoch 92/100
 - 0s - loss: 0.0955 - acc: 0.9642
Epoch 93/100
 - 0s - loss: 0.0942 - acc: 0.9664
Epoch 94/100
 - 0s - loss: 0.0884 - acc: 0.9630
Epoch 95/100
 - 0s - loss: 0.0785 - acc: 0.9709
Epoch 96/100
 - 0s - loss: 0.0866 - acc: 0.9698
Epoch 97/100
 - 0s - loss: 0.0904 - acc: 0.9686
Epoch 98/100
 - 0s - loss: 0.0748 - acc: 0.9698
Epoch 99/100
 - 0s - loss: 0.0793 - acc: 0.9686
Epoch 100/100
 - 0s - loss: 0.0712 - acc: 0.9731
Epoch 1/100
 - 4s - loss: 1.8941 - acc: 0.2307
Epoch 2/100
 - 0s - loss: 2.3535 - acc: 0.3550
Epoch 3/100
 - 0s - loss: 2.4434 - acc: 0.3135
Epoch 4/100
 - 0s - loss: 1.7659 - acc: 0.3639
Epoch 5/100
 - 0s - loss: 1.4318 - acc: 0.3841
Epoch 6/100
 - 0s - loss: 1.3472 - acc: 0.3863
Epoch 7/10

Epoch 58/100
 - 0s - loss: 0.1176 - acc: 0.9597
Epoch 59/100
 - 0s - loss: 0.1073 - acc: 0.9631
Epoch 60/100
 - 0s - loss: 0.1235 - acc: 0.9530
Epoch 61/100
 - 0s - loss: 0.1251 - acc: 0.9541
Epoch 62/100
 - 0s - loss: 0.1247 - acc: 0.9541
Epoch 63/100
 - 0s - loss: 0.1530 - acc: 0.9530
Epoch 64/100
 - 0s - loss: 0.1198 - acc: 0.9609
Epoch 65/100
 - 0s - loss: 0.1178 - acc: 0.9631
Epoch 66/100
 - 0s - loss: 0.1064 - acc: 0.9586
Epoch 67/100
 - 0s - loss: 0.1038 - acc: 0.9687
Epoch 68/100
 - 0s - loss: 0.1040 - acc: 0.9564
Epoch 69/100
 - 0s - loss: 0.1358 - acc: 0.9631
Epoch 70/100
 - 0s - loss: 0.1103 - acc: 0.9575
Epoch 71/100
 - 0s - loss: 0.1046 - acc: 0.9631
Epoch 72/100
 - 0s - loss: 0.0951 - acc: 0.9698
Epoch 73/100
 - 0s - loss: 0.1107 - acc: 0.9664
Epoch 74/100
 - 0s - loss: 0.0971 - acc: 0.9698
Epoch 75/100
 - 0s - loss: 0.0764 - acc: 0.9743
Epoch 76/100
 - 0s - loss: 0.0956 - acc: 0.9664
Epoch 77/100
 - 0s - loss: 0.0787 - acc: 0.9687
Epoch 78/100
 - 0s - loss: 0.1120 - acc:

In [9]:
t = grid_result.best_estimator_

In [10]:
print(t.score(X_test, y_test_for_keras))

0.6785714030265808


In [11]:
t.model.save('../model/Keras_best_params_pos_fs.h5')

In [12]:
# tf.set_random_seed(42)

# # make keras model
# start = time.time()
# inputs = Input(shape=(X_train.shape[1],))

# x = Dense(512, activation='relu')(inputs)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# predictions = Dense(
#     len(df['Subclass'].value_counts()), activation='softmax')(x)
# model = Model(inputs=inputs, outputs=predictions)

# # compile
# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=Adamax(),
#     metrics=['accuracy']
# )

# epochs = 100
# batch_size = 1000
# es = EarlyStopping(monitor='val_loss', patience=20)

# history = model.fit(
#     X_train,
#     y_train_for_keras,
#     epochs=epochs,
#     batch_size=batch_size,
#     validation_data=(X_test, y_test_for_keras),
#     verbose=1,
#     callbacks=[
#         es,
#     ]
# )

# model.save('../model/Keras_fs_pos.h5')